In [1]:
import pandas as pd
import openpyxl
import xlsxwriter
import numpy as np
import pdb
from dateutil.relativedelta import *
import glob
import matplotlib.pyplot as plt

In [2]:
#Converting xls data
shortlisted_companies_df  = pd.DataFrame() 
for input_file  in glob.glob('../../finance_vicky/files/output/backtest/*.xlsx'):
    xl = pd.ExcelFile(input_file)
    if 'Shortlisted Companies' in xl.sheet_names:
        script_df = xl.parse('Shortlisted Companies')
        shortlisted_companies_df = pd.concat([shortlisted_companies_df, script_df])       
shortlisted_companies_df.reset_index(inplace = True)

shortlisted_companies_df.set_index(['Start Date', 'Script Name'], inplace=True)
shortlisted_companies_df.sort_index(level='Start Date', inplace=True)

In [4]:

print(shortlisted_companies_df.head())
simulated_df = pd.DataFrame()
capital_df = pd.DataFrame()
capital = 10000000
capital_count = 1

#distribute the money equally across all scripts.
for i, data in shortlisted_companies_df.groupby(level = 0):
    sc_df = pd.DataFrame()
    sc_df = pd.concat([sc_df,data.loc[(i)]])
    total_scripts = len(sc_df.index)
    share_per_script = capital/total_scripts 
    if share_per_script > 100:
        for script in sc_df.index.values:
            shares = np.floor(share_per_script/sc_df.loc[script, 'Open for Test Data'])
            sc_df.loc[script, 'No. of Shares'] = shares
            net_pl = shares * sc_df.loc[script, 'Net for Test Data']
            sc_df.loc[script, 'Investment'] = shares * sc_df.loc[script, 'Open for Test Data']
            sc_df.loc[script, 'Balance'] = share_per_script - shares * sc_df.loc[script, 'Open for Test Data']
            sc_df.loc[script, 'Disinvestment'] = shares * sc_df.loc[script, 'Close for Test Data']
            sc_df.loc[script,'net_pl'] = net_pl
            sc_df.loc[script, 'Start Date'] = i
            capital += sc_df.loc[script, 'net_pl']
            capital += sc_df.loc[script, 'Balance']         
        sc_df.reset_index(inplace=True)
        simulated_df = pd.concat([simulated_df, sc_df])
     
        
        df = pd.DataFrame.from_records([{'Invest Date': i, 'Investment': sc_df['Investment'].sum(), 'Gains/Loss' : sc_df['net_pl'].sum(),
                      'Disinvestment': sc_df['Disinvestment'].sum(), 'Balance': sc_df['Balance'].sum(),
                      'Total Funds': sc_df['Disinvestment'].sum() + sc_df['Balance'].sum(), 
                      'Disinvest Date': sc_df['End Date'][0]}], index='Invest Date')
        
        capital_df = pd.concat([capital_df, df])            
    else: 
        capital += 10000000
        capital_count += 1
print(capital, capital_count)
print(capital_df.head())
#print(simulated_df.columns)
#simulated_df.set_index(['Start Date', 'Script Name'], inplace=True)
#simulated_df.sort_index(level='Start Date', inplace=True)
#print(simulated_df.head())

                             index  Positive  Negative    Net     Open  \
Start Date Script Name                                                   
2007-04-03 Aban Offshore         2    175.98   -100.18  74.07  1139.35   
           Aditya Bir. Nuv.      3    131.41    -90.66  40.98   348.31   
           BF Utilities          1    193.21    -99.07  91.04  1092.85   
           Centurion BnkP        5    134.80    -91.26  41.23    25.75   
           Guj.St.Petronet       8    139.57   -101.86  37.83    37.10   

                               Close  Market Cap      P/YR  \
Start Date Script Name                                       
2007-04-03 Aban Offshore     2052.45     4204.20  2.247782   
           Aditya Bir. Nuv.   489.08     4484.41  3.474320   
           BF Utilities      2125.45     4115.67  2.058348   
           Centurion BnkP      35.60     3626.37  3.614213   
           Guj.St.Petronet     48.55     2011.71  4.240175   

                             Open for Test Dat

In [5]:

output_file = '../../finance_vicky/files/output/backtest/simulation_details.xlsx'
writer = pd.ExcelWriter(output_file, engine='xlsxwriter', datetime_format='mm/dd/yy', date_format='mm/dd/yy')
#pdb.set_trace()
simulated_df.set_index(['Start Date', 'Script Name'], inplace = True)
simulated_df.sort_index(level='Start Date', inplace=True)
simulated_df.to_excel(writer,sheet_name='Sheet') 
capital_df.to_excel(writer, sheet_name='Capital Gains')
writer.save()

In [19]:
#capital_df.head()
capital_df.dropna(inplace=True)
if 'index' in capital_df.columns:
    capital_df.drop(['index'], axis=1, inplace=True)
capital_df.sort_index(inplace=True)
#capital_df.head(39)


In [ ]:
plt.plot(capital_df['Gains/Loss'])
plt.show()

In [30]:
simulated_df.head()
simulated_df_groups = simulated_df.groupby(['Script Name'])
for key, item in simulated_df_groups:
    print(simulated_df_groups.get_group(key), "\n\n")

     level_0 Start Date   Script Name  index  Positive  Negative   Net  \
201      201 2012-01-03  ABG Shipyard      4     85.93    -89.02 -4.51   

       Open   Close  Market Cap       P/YR  Open for Test Data  \
201  425.75  389.35     2167.92 -10.696429              389.35   

     Close for Test Data  Net for Test Data   End Date  No. of Shares  \
201                381.6              -7.75 2012-04-30         4468.0   

     Investment     Balance  Disinvestment   net_pl  
201   1739615.8  292.156096      1704988.8 -34627.0   


    level_0 Start Date Script Name  index  Positive  Negative   Net    Open  \
66       66 2008-10-01   AIA Engg.      0    140.98   -133.65  7.33  265.18   

     Close  Market Cap       P/YR  Open for Test Data  Close for Test Data  \
66  261.17     2492.69 -65.129676              261.17               114.05   

    Net for Test Data   End Date  No. of Shares  Investment  Balance  \
66            -147.12 2009-01-30         3785.0   988528.45   24.075   


     level_0 Start Date Script Name  index  Positive  Negative    Net   Open  \
264      264 2015-07-01  Bajaj Corp      8    107.38    -42.48  66.05  240.3   

     Close  Market Cap      P/YR  Open for Test Data  Close for Test Data  \
264  438.5     3544.43  2.212412               438.5               409.05   

     Net for Test Data   End Date  No. of Shares  Investment     Balance  \
264             -29.45 2015-10-30         2982.0   1307607.0  336.986537   

     Disinvestment   net_pl  
264      1219787.1 -87819.9   


     level_0 Start Date       Script Name  index  Positive  Negative    Net  \
180      180 2011-07-01  Bajaj Electrical      9     91.36    -66.52  17.76   

      Open   Close  Market Cap      P/YR  Open for Test Data  \
180  217.1  244.35     2117.81  8.966972              244.35   

     Close for Test Data  Net for Test Data   End Date  No. of Shares  \
180               195.45              -48.9 2011-10-31         7630.0   

     Investment    Balance  Disin

     level_0 Start Date    Script Name  index  Positive  Negative    Net  \
386      386 2017-10-03  Chambal Fert.      6     97.54     -2.24  94.82   
409      409 2018-01-02  Chambal Fert.     11     92.39     -3.90  87.53   
427      427 2018-04-03  Chambal Fert.      7    108.13    -30.37  73.58   

      Open   Close  Market Cap      P/YR  Open for Test Data  \
386  60.20  140.80     2505.58  1.746898              140.80   
409  71.45  156.85     2973.82  1.836651              156.85   
427  87.05  165.50     3623.11  2.109624              165.50   

     Close for Test Data  Net for Test Data   End Date  No. of Shares  \
386               160.40               19.6 2018-01-31        12588.0   
409               196.35               39.5 2018-04-30        11590.0   
427               147.00              -18.5 2018-07-31        13476.0   

     Investment     Balance  Disinvestment    net_pl  
386   1772390.4   34.769795      2019115.2  246724.8  
409   1817891.5   70.525081      22

     level_0 Start Date     Script Name  index  Positive  Negative    Net  \
79        79 2009-01-01  Fortis Health.      1    188.21   -214.28 -26.07   
143      143 2010-07-01  Fortis Health.      9    145.50    -87.79  57.71   

      Open   Close  Market Cap      P/YR  Open for Test Data  \
79   113.2   71.78     2509.24 -1.732979               71.78   
143   93.1  154.10     2063.83  2.526230              154.10   

     Close for Test Data  Net for Test Data   End Date  No. of Shares  \
79                 69.59              -2.19 2009-04-29         8043.0   
143               160.15               6.05 2010-10-29        12929.0   

     Investment     Balance  Disinvestment    net_pl  
79    577326.54   48.897000      559712.37 -17614.17  
143  1992358.90  119.987001     2070579.35  78220.45   


     level_0 Start Date      Script Name  index  Positive  Negative     Net  \
327      327 2016-07-01  Future Consumer      5    114.70    -54.25   63.26   
413      413 2018-01-02  Futu

     level_0 Start Date     Script Name  index  Positive  Negative    Net  \
268      268 2015-07-01  Hexaware Tech.     11    116.53    -61.83  57.88   

      Open  Close  Market Cap      P/YR  Open for Test Data  \
268  155.4  257.2     4666.66  2.526523               257.2   

     Close for Test Data  Net for Test Data   End Date  No. of Shares  \
268               239.75             -17.45 2015-10-30         5085.0   

     Investment    Balance  Disinvestment    net_pl  
268   1307862.0  81.986537     1219128.75 -88733.25   


   level_0 Start Date  Script Name  index  Positive  Negative    Net  Open  \
5        5 2007-04-03  Hind.Copper      4    147.74   -100.94  47.09  61.2   

   Close  Market Cap      P/YR  Open for Test Data  Close for Test Data  \
5  85.15     4456.71  3.555324               85.15               101.85   

   Net for Test Data   End Date  No. of Shares  Investment  Balance  \
5               16.7 2007-07-31        11743.0   999916.45    83.55   

   Disinv

     level_0 Start Date     Script Name  index  Positive  Negative    Net  \
197      197 2011-10-03  Jubilant Food.      4    135.15    -76.05  62.15   
235      235 2012-10-03  Jubilant Food.      4    114.67    -48.42  66.25   
245      245 2013-01-02  Jubilant Food.      7     83.68    -20.88  63.19   

       Open   Close  Market Cap      P/YR  Open for Test Data  \
197  240.83  396.73     3098.94  2.544772              376.98   
235  376.98  662.28     4875.79  2.321346              662.28   
245  375.58  661.93     4877.22  2.311612              661.93   

     Close for Test Data  Net for Test Data   End Date  No. of Shares  \
197               470.03              93.05 2012-01-31         4423.0   
235               600.03             -62.25 2013-01-31         4063.0   
245               554.18            -107.75 2013-04-30         4266.0   

     Investment     Balance  Disinvestment     net_pl  
197  1667382.54  219.900043     2078942.69  411560.15  
235  2690843.64  460.2967

     level_0 Start Date  Script Name  index  Positive  Negative     Net  \
393      393 2017-10-03  Minda Inds.      1    122.36    -15.92  106.13   
419      419 2018-01-02  Minda Inds.      2    128.33     16.56  145.69   
437      437 2018-04-03  Minda Inds.      3    120.38    -17.00  102.88   

       Open   Close  Market Cap      P/YR  Open for Test Data  \
393  104.18  281.35     2480.08  1.588023              281.35   
419  104.85  423.96     2495.95  1.328570              423.96   
437  145.87  381.21     3780.86  1.619827              381.21   

     Close for Test Data  Net for Test Data   End Date  No. of Shares  \
393               411.68             130.33 2018-01-31         6299.0   
419               363.50             -60.46 2018-04-30         4288.0   
437               403.05              21.84 2018-07-31         5850.0   

     Investment     Balance  Disinvestment     net_pl  
393  1772223.65  201.519795     2593172.32  820948.67  
419  1817940.48   21.545081     1

    level_0 Start Date  Script Name  index  Positive  Negative    Net   Open  \
39       39 2008-01-02  Patel Engg.      8    156.04    -65.13  92.34  451.1   

     Close  Market Cap      P/YR  Open for Test Data  Close for Test Data  \
39  992.95     2693.07  1.832518              992.95                567.3   

    Net for Test Data   End Date  No. of Shares  Investment   Balance  \
39            -425.65 2008-04-30         2143.0  2127891.85  729.8875   

    Disinvestment     net_pl  
39      1215723.9 -912167.95   


     level_0 Start Date    Script Name  index  Positive  Negative    Net  \
247      247 2013-01-02  Phoenix Mills      8     80.65    -28.06  51.92   
260      260 2013-04-02  Phoenix Mills     11     47.21    -21.03  25.97   

       Open   Close  Market Cap      P/YR  Open for Test Data  \
247  167.15  264.55     2421.17  2.716119              264.55   
260  204.55  252.65     2962.91  5.252599              252.65   

     Close for Test Data  Net for Test Data   E

     level_0 Start Date    Script Name  index  Positive  Negative    Net  \
84        84 2009-01-01  Sterling Intl      9    139.26    -77.67  61.59   
94        94 2009-04-01  Sterling Intl      9    141.17   -112.80  24.95   
176      176 2011-04-01  Sterling Intl      8    111.93    -72.31  41.15   
189      189 2011-07-01  Sterling Intl      1    137.78    -79.55  59.76   

       Open   Close  Market Cap       P/YR  Open for Test Data  \
84   134.55  211.15     2513.39   2.756527              211.15   
94   181.75  193.00     3395.09  17.155556              193.00   
176  178.50  243.95     4844.49   3.727273              243.95   
189  152.50  245.15     4138.85   2.645979              245.15   

     Close for Test Data  Net for Test Data   End Date  No. of Shares  \
84                171.05             -40.10 2009-04-29         2734.0   
94                139.65             -53.35 2009-07-31         2851.0   
176               222.90             -21.05 2011-07-29         7291.0

    level_0 Start Date Script Name  index  Positive  Negative    Net    Open  \
30       30 2007-10-03         UPL     11    111.15    -58.79  54.24  122.38   

     Close  Market Cap      P/YR  Open for Test Data  Close for Test Data  \
30  196.53     4581.72  2.650438              196.53               174.33   

    Net for Test Data   End Date  No. of Shares  Investment    Balance  \
30              -22.2 2008-01-31         9642.0  1894942.26  88.035833   

    Disinvestment    net_pl  
30     1680889.86 -214052.4   


   level_0 Start Date Script Name  index  Positive  Negative     Net   Open  \
8        8 2007-04-03     Unitech      0    259.52    -28.77  226.77  22.49   

    Close  Market Cap     P/YR  Open for Test Data  Close for Test Data  \
8  174.83     3652.95  1.14763              174.83               279.43   

   Net for Test Data   End Date  No. of Shares  Investment  Balance  \
8              104.6 2007-07-31         5719.0   999852.77   147.23   

   Disinvestment   